### Build a Arc-I Model

<img src="https://raw.githubusercontent.com/NTMC-Community/MatchZoo/2.0/artworks/matchzoo-logo.png" alt="logo" style="width:600px;float: center"/>

This is a tutorial on training *Arc-I Model* [Hu et al. 2014](http://papers.nips.cc/paper/5550-convolutional-neural-network-architectures-for-matching-natural-language-sentences.pdf) model with [MatchZoo](https://github.com/faneshion/MatchZoo). We use [WikiQA](https://aclweb.org/anthology/D15-1237) as the example benchmark data set to show the usage.

*To walk through this notebook, you need approx 90 minutes.*

-------

**TL;DR**

The following code block illustrates the main workflow of how to train a Arc-I model. 

```python
from matchzoo import preprocessor
from matchzoo import generators
from matchzoo import models

train, test = ... # prepare your training data and test data.

arci_preprocessor = preprocessor.ArcIPreprocessor()
processed_tr = arci_preprocessor.fit_transform(train, stage='train')
processed_te = arci_preprocessor.fit_transform(test, stage='test')


generator_tr = generators.PointGenerator(processed_tr)
generator_te = generators.PointGenerator(processed_te)
# Example, train with generator, test with the first batch.
X_te, y_te = generator_te[0]

arci_model = models.ArcIModel()
arci_model.guess_and_fill_missing_params()
arci_model.build()
arci_model.compile()
arci_model.fit_generator(generator_tr)
# Make predictions
predictions = arci_model.predict([X_te.text_left, X_te.text_right])
```

-----

MatchZoo expect a list of *Quintuple* as training data:

```python
train = [('qid0', 'did0', 'query 0', 'document 0', 'label 0'),
         ('qid0', 'did1', 'query 0', 'document 1', 'label 1'),
          ...,
         ('qid1', 'did2', 'query 1', 'document 2', 'label 3')]
```

The corresponded columns are `(text_left_id, text_right_id, text_left, text_right, label)`. For Information Retrieval task, *text_left* is referred as *query*, and *text_right* is document.

For the test case, MatchZoo expect a list of *Quadruple* (we do not need labels) as input:

```python
test = [('qid9', 'did5', 'query 9', 'document 5'),
         ...,
        ('qid2', 'did7', 'query 2', 'document 7')]
```

### Table of Content

+ Prepare WikiQA dataset
    - Download
    - Load
    - Adjustment
+ Preprocessing
+ Data Generator
+ Model Training
    - Initialize
    - Hyper-Parameters
    - Make Prediction
    - Model Persistence
- Reference

### Prepare WikiQA dataset

#### Download

We take WikiQA as the example benchmark dataset to show the usage of MatchZoo. Firstly you need to downlowd the data and uncompress the data, we provided the following script to help you download the dataset into `MatchZoo/data/WikiQA` folder, you can change the directory in the following script.

If you already have WikiQA dataset downloaded on your machine, skip the following script.

In [61]:
!rm -rf ../../data/WikiQA
!mkdir -p ../../data/WikiQA
!wget -P ../../data/WikiQA https://download.microsoft.com/download/E/5/F/E5FCFCEE-7005-4814-853D-DAA7C66507E0/WikiQACorpus.zip
!unzip -d ../../data/WikiQA ../../data/WikiQA/WikiQACorpus.zip

--2018-10-20 21:23:13--  https://download.microsoft.com/download/E/5/F/E5FCFCEE-7005-4814-853D-DAA7C66507E0/WikiQACorpus.zip
正在解析主机 download.microsoft.com (download.microsoft.com)... 61.240.136.46, 221.194.130.172
正在连接 download.microsoft.com (download.microsoft.com)|61.240.136.46|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：7094233 (6.8M) [application/octet-stream]
正在保存至: “../../data/WikiQA/WikiQACorpus.zip”

WikiQACorpus.zip    100%[===================>]   6.76M   421KB/s  用时 14s       

2018-10-20 21:23:27 (512 KB/s) - 已保存 “../../data/WikiQA/WikiQACorpus.zip” [7094233/7094233])

Archive:  ../../data/WikiQA/WikiQACorpus.zip
   creating: ../../data/WikiQA/WikiQACorpus/emnlp-table/
  inflating: ../../data/WikiQA/WikiQACorpus/emnlp-table/WikiQA.CNN.dev.rank  
  inflating: ../../data/WikiQA/WikiQACorpus/emnlp-table/WikiQA.CNN.test.rank  
  inflating: ../../data/WikiQA/WikiQACorpus/emnlp-table/WikiQA.CNN-Cnt.dev.rank  
  inflating: ../../data/WikiQA/WikiQACorpus/emnlp-table/WikiQA.CNN-Cnt.

#### Load & Adjustment

The *train/dev/test* files of WikiQA are *WikiQA-train.tsv*, *WikiQA-dev.tsv*, *WikiQA-test.tsv* under the uncompressed folder WikiQACorpus. The data format of WikiQA is as follows:

`QuestionID\tQuestion\tDocumentID\tDocumentTitle\tSentenceID\tSentence\tLabel`

We can convert this format to the expected input format of MatchZoo.

In [1]:
data_folder = '../../data/WikiQA/WikiQACorpus/'

def read_data(input, stage):
    output_list = []
    index = 0
    with open(input) as fin:
        for l in fin:
            tok = l.split('\t')
            if index != 0:
                if stage == 'test':
                    output_list.append((tok[0], tok[4], tok[1], tok[5])) # qid, did, q, d, label
                else:
                    output_list.append((tok[0], tok[4], tok[1], tok[5], tok[6])) # qid, did, q, d 
            index += 1
    return output_list

train = read_data(data_folder + 'WikiQA-train.tsv', stage='train')
dev   = read_data(data_folder + 'WikiQA-dev.tsv', stage='dev')
test  = read_data(data_folder + 'WikiQA-test.tsv', stage='test')

### Preprocessing


#### Download

We first download the GLoVe pre-trained word embeddings. Be pationt - this takes a long time related to your network service. Alternatively, you can download it via another downloading tool and put **glove.840B.300d.txt** in the directory **../../data/embedding**.

In [50]:
!mkdir -p ../../data/embedding
!wget -O ../../data/embedding/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip 
!unzip -o ../../data/embedding/glove.840B.300d.zip -d ../../data/embedding

--2018-10-20 20:20:26--  http://nlp.stanford.edu/data/glove.840B.300d.zip
正在解析主机 nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
正在连接 nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... 已连接。
已发出 HTTP 请求，正在等待回应... 302 Found
位置：https://nlp.stanford.edu/data/glove.840B.300d.zip [跟随至新的 URL]
--2018-10-20 20:20:27--  https://nlp.stanford.edu/data/glove.840B.300d.zip
正在连接 nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：2176768927 (2.0G) [application/zip]
正在保存至: “../../data/embedding/glove.840B.300d.zip.5”

glove.840B.300d.zip 100%[===================>]   2.03G   535KB/s  用时 52m 54s   

2018-10-20 21:13:22 (670 KB/s) - 已保存 “../../data/embedding/glove.840B.300d.zip.5” [2176768927/2176768927])

Archive:  ../../data/embedding/glove.840B.300d.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comm

#### Preproceesing with pre-trained embeddings

We run the processor in three lines of code.  
If we use **the randomly initialized embeddings**, just keep default value for *embedding_file*, and the preprocessing code is as follows:
```python
from matchzoo import preprocessor
arci_preprocessor = preprocessor.ArcIPreprocessor()
processed_tr = arci_preprocessor.fit_transform(train, stage='train')
processed_te = arci_preprocessor.fit_transform(test, stage='test')
```

In [2]:
# Initialize a dssm preprocessor.
from matchzoo import preprocessor
arci_preprocessor = preprocessor.ArcIPreprocessor(embedding_file='../../data/embedding/glove.840B.300d.txt')
processed_tr = arci_preprocessor.fit_transform(train, stage='train')
processed_te = arci_preprocessor.fit_transform(test, stage='test')

Using TensorFlow backend.
Start building vocabulary & fitting parameters.
2118it [00:00, 3248.06it/s]
18841it [00:08, 2200.22it/s]
Some words are not shown in term_index(29924). Total number of words are 29925.
2196016it [01:05, 33488.75it/s]
Start processing input data for train stage.
2118it [00:00, 2464.37it/s]
18841it [00:10, 1754.62it/s]
Start processing input data for test stage.
633it [00:00, 2482.71it/s]
5961it [00:03, 1895.94it/s]


**What is `processed_tr`?**

`processed_tr` is a **MatchZoo DataPack** data structure (see `matchzoo/datapack.py`). It contains 
1. A *2-columns* `pandas DataFrame` called `left` to host all the pre-processed records including index and processed text to store `text_left` and `id_left`.
2. A *2-columns* `pandas DataFrame` called `right` to host all the pre-processed records including index and processed text to store `text_right` and `id_right`.
3. A *2-columns* `pandas DataFrame` called `relation` to host all the pre-processed records including index and index mapping `id_left` and `id_right`.
4. A `context` property (dictionary) consists of all the parameters fitted during pre-processing. 


The `fit_transform` method is a sequential combination of two methods:

1. Fit parameters using the `fit` function, this only happens when `stage='train'`.
2. Transform data into expected format.

So the previous preprocessing code in test stage can also be written as:

```python
processed_te = dssm_preprocessor.transform(test, stage='test')
```

As described, the fitted parameters were stored in `context` property, to access the context, just call:

```python
print(processed_tr.context)
```
An example:

In [3]:
print('vocab size: ', len(processed_tr.context['term_index']))

vocab size:  29924


**What has been stored in the `context?`** 

We stored `term_index` and `input_shapes` in the context property. 


**What is `arci_preprocessor` actually doing?**

The `arci_preprocessor` is calling a sequence of `process_units`. Each `process_unit` is designed to perform one atom operation on input data. For instance, in `arci_preprocessor`, we called:


1. TokenizeUnit: Perform tokenization on raw input data.
2. LowercaseUnit: Transform all tokens into lower case.
3. PuncRemovalUnit: Remove all the punctuations.
4. StopRemovalUnit: Remove all the stopwords.

Depending on whether using the pre-trained embeddings, the preprocessor loads the embedding from an embedding file or randomly initializes the embeddings.


----

### Data Generation

For memory efficiency, we expect you to use **generator** to generate batches of data on the fly. For example, we can create a **PointGenerator** as follows:

In [4]:
from matchzoo import generators
from matchzoo import tasks
generator_tr = generators.PointGenerator(inputs=processed_tr, task=tasks.Ranking(), batch_size=64, stage='train')
generator_te = generators.PointGenerator(inputs=processed_te, task=tasks.Ranking(), batch_size=64, stage='test')

To get the first batch of trainig data, just call `X_train, y_train = generator[0]`.

**What is PointGenerator?**
**PointGenerator** is this case, it is assumed that each query-document pair in the training data has a numerical or ordinal score. Then the problem can be approximated by a regression/Classification problem — given a single query-document pair, predict its score.

A number of existing supervised machine learning algorithms can be readily used for this purpose. Ordinal regression and classification algorithms can also be used in pointwise approach when they are used to predict the score of a single query-document pair, and it takes a small, finite number of values.

**What is PairGenerator?**
In this case, the problem is approximated by a classification problem — learning a binary classifier that can tell which document is better in a given pair of documents.

In MatchZoo, **PairGenerator** generate one positive & `num_neg` negative examples per pair. As an example, to train a DSSM model (for document ranking), we use `num_neg=4`. 

**What is ListGenerator?**
This generator try to directly optimize the value of evaluation measures, averaged over all queries in the training data. 

Chosse the appropriate generator based on your `task`.

----

### Train Your Arc-I Model

To train a Arc-I model, we need to create an instance of ArcIModel:

In [19]:
from matchzoo import models
arci_model = models.ArcIModel()

Then, we need to set hyper-parameters to our Arc-I Model. In general, there are **two types of hyper-parameters**:

**Required parameters**: For Arc-I, if you are using the pre-trained embeddings, you're required to set the embedding_mat and the size manually!

In [51]:
# The fitted parameters is stored in the `context` property of pre-processor instance during the training stage.
from matchzoo import losses
from matchzoo import tasks
arci_model.params['task'] = tasks.Ranking()
arci_model.params['embedding_dim'] = processed_tr.context['embedding_mat'].shape[1]
arci_model.params['vocab_size'] = processed_tr.context['embedding_mat'].shape[0]
arci_model.params['embedding_mat'] = processed_tr.context['embedding_mat']

**Tunable parameters**: For Arc-I, you're allowed to tune these parameters:

```python
from matchzoo import tasks

        
params = {
            'optimizer': 'adam' # By default, we use sgd. See keras optimizer.
            'trainable_embedding': False # Whether finetune the embeddings.
            'num_blocks': 1 # Number of the convolution layers.
            'left_kernel_count': [32] # Number of filters in convolution. See keras Conv1D.
            'left_kernel_size': [3] # The length of the 1D convolution window. See keras Conv1D.
            'right_kernel_count': [32] # Number of filters in convolution. See keras Conv1D.
            'right_kernel_size': [3] # The length of the 1D convolution window. See keras Conv1D.
            'activation': 'relu' # Activation function in convolution. See keras Conv1D.
            'left_pool_size': [2] # Size of the max pooling windows for the left text. See keras MaxPooling1D.
            'right_pool_size': [2] # Size of the max pooling windows for rhe right text. See keras MaxPooling1D.
            'padding': 'same' # Padding mode. See keras padding in Conv1D.   
            'dropout_rate': 0.0 # Probability of an element to be zeroed. See keras Dropout.
            'embedding_random_scale': 0.2 # The range of the random initialized embedding. 
            'task': tasks.Classification, # Default Classification, you can use tasks.Ranking
            'loss': 'categorical_crossentropy', # categorical_crossentropy, see keras loss.
            'metric': 'acc', # Accuracy by default, see keras metric.
         }
```

In [32]:
arci_model.guess_and_fill_missing_params()
print('arci parameters: ', arci_model.params)

arci parameters:  name                          ArcIModel
model_class                   <class 'matchzoo.models.arci_model.ArcIModel'>
input_shapes                  [(32,), (32,)]
task                          <matchzoo.tasks.ranking.Ranking object at 0x103778400>
metrics                       ['mae']
loss                          mse
optimizer                     adam
trainable_embedding           False
embedding_dim                 100
vocab_size                    29925
num_blocks                    1
left_kernel_count             [32]
left_kernel_size              [3]
right_kernel_count            [32]
right_kernel_size             [3]
activation                    relu
left_pool_size                [2]
right_pool_size               [2]
padding                       same
dropout_rate                  0.0
embedding_mat                 [[ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.01879   0.044005 -0.19154  ... -0.36637   0.58809  -0.098892]
 [-0.20923  -0.3

#### Model Training

To train the model after all the parameters were settled, call:

In [33]:
arci_model.build()
arci_model.compile()
# Fit the arci model on generator.
arci_model.fit_generator(generator_tr, steps_per_epoch=200, epochs=10)
# Make predictions on the first batch of test data
X_te, y_te = generator_te[0]
predictions = arci_model.predict([X_te.text_left, X_te.text_right])

Epoch 1/10
200/200 [==============================] - 3s 17ms/step - loss: 0.0676 - mean_absolute_error: 0.1553
Epoch 2/10
200/200 [==============================] - 3s 17ms/step - loss: 0.0480 - mean_absolute_error: 0.1086
Epoch 3/10
200/200 [==============================] - 3s 16ms/step - loss: 0.0483 - mean_absolute_error: 0.1079
Epoch 4/10
200/200 [==============================] - 3s 17ms/step - loss: 0.0412 - mean_absolute_error: 0.0996
Epoch 5/10
200/200 [==============================] - 4s 18ms/step - loss: 0.0412 - mean_absolute_error: 0.1024
Epoch 6/10
200/200 [==============================] - 3s 15ms/step - loss: 0.0403 - mean_absolute_error: 0.1058
Epoch 7/10
200/200 [==============================] - 3s 15ms/step - loss: 0.0377 - mean_absolute_error: 0.1027
Epoch 8/10
200/200 [==============================] - 3s 15ms/step - loss: 0.0384 - mean_absolute_error: 0.1047
Epoch 9/10
200/200 [==============================] - 3s 15ms/step - loss: 0.0331 - mean_absolute_error:

#### Make Prediction

In [53]:
for id_left, id_right, pred in zip(X_te.id_left, X_te.id_right, predictions):
    print("{}/{} is predicted as {}".format(id_left, id_right, pred))

Q2714/D2530-6 is predicted as [-0.07276025]
Q1338/D1278-8 is predicted as [0.00079855]
Q72/D72-2 is predicted as [0.080437]
Q1409/D1342-5 is predicted as [0.05105157]
Q971/D937-8 is predicted as [0.04178461]
Q2466/D2310-1 is predicted as [0.1102946]
Q1675/D1591-11 is predicted as [-0.09402043]
Q1021/D984-4 is predicted as [0.07721019]
Q1157/D1112-1 is predicted as [0.0401497]
Q2623/D2448-8 is predicted as [0.01702134]
Q144/D144-3 is predicted as [-0.05467525]
Q1211/D1160-13 is predicted as [0.21775976]
Q2892/D1077-4 is predicted as [-0.03775209]
Q1434/D1365-5 is predicted as [0.00870322]
Q2839/D2636-0 is predicted as [-0.02167417]
Q2931/D2712-9 is predicted as [-0.1458192]
Q1416/D1349-23 is predicted as [0.01549989]
Q309/D308-10 is predicted as [0.04581261]
Q897/D867-0 is predicted as [0.50736415]
Q91/D91-8 is predicted as [0.08594398]
Q2184/D2058-12 is predicted as [0.13852651]
Q844/D818-16 is predicted as [0.03765132]
Q1211/D1160-11 is predicted as [-0.01216602]
Q715/D696-9 is predic

#### Model Persistence

You can persist your trained model using `model.save()` and `load_model` function:

```python
from matchzoo import engine
# Save the model to dir.
arci_model.save('/your-model-saved-path')
# And load the model from dir.
engine.load_model('/your-model-saved-path')
```

## Reference

[Hu et al. 2014] Baotian Hu, Zhengdong Lu, Hang Li, and Qingcai Chen. "Convolutional neural network architectures for matching natural language sentences." In Advances in neural information processing systems (NIPS), pp. 2042-2050. 2014.